In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch 
import torch.nn as nn 
from torch.utils.data import DataLoader, Subset

from train import * 
from src.autoencoder import * 
from Data.two_body_dataset import *
from src.gan import *
from src.NN.multi_step_pred import *
from src.LaLiGAN.custom_emlp_laligan import *
from src.NN.custom_emlp_multistep import *

import warnings
warnings.filterwarnings("ignore", message="torch.utils._pytree._register_pytree_node is deprecated")

/mnt/d_disk/ch20b065/.conda/envs/liegan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/d_disk/ch20b065/.conda/envs/liegan/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
input_timesteps = 4
output_timesteps = 2
n_dim = 8

train_dataset = TwoBodyDataset(
    save_path='./data/two_body_dataset.pkl',
    input_timesteps = input_timesteps,
    output_timesteps = output_timesteps,
    mode='train'
)

test_dataset = TwoBodyDataset(
    save_path='./data/two_body_dataset.pkl',
    input_timesteps = input_timesteps,
    output_timesteps = output_timesteps,
    mode='test'
)



# Get the actual size of the dataset
train_size = 2000
test_size = 400

# Sample accordingly
train_indices = np.random.choice(len(train_dataset), size=train_size, replace=False)
test_indices = np.random.choice(len(test_dataset), size=test_size, replace=False)


train_dataset = Subset(train_dataset,train_indices)
test_dataset = Subset(test_dataset,test_indices)


In [3]:
class Args:
    def __init__(self):
        # model & training settings
        self.batch_size = 64
        self.num_epochs = 100
        self.lr_d = 2e-4
        self.lr_g = 1e-3
        self.reg_type = 'cosine'
        self.lamda = 1e-2
        self.p_norm = 2
        self.droprate_init = 0.8
        self.mu = 0.0
        self.activate_threshold = False
        self.D_loss_threshold = 0.25
        self.model = 'lie'
        self.coef_dist = 'normal'
        self.g_init = 'random'
        self.sigma_init = 1
        self.uniform_max = 1
        self.normalize_Li = False
        self.augment = False
        self.n_channel = 1
        self.eta = 1.0
        self.incremental = False
        
        # dataset settings
        self.task = 'pendulum'
        self.dataset_name = 'pendulum'
        self.dataset_config = None
        self.dataset_size = 2000
        self.x_type = 'vector'
        self.y_type = 'vector'
        self.input_timesteps = 1
        self.output_timesteps = 1
        self.n_component = 1
        self.noise = 0.0
        
        # run settings
        self.seed = 42
        self.gpu = 0
        self.print_every = 1
        self.save_path = 'saved_model'
        self.save_name = 'default'
        self.device = get_device()

        # regressor

# Create the args object with default values
args = Args()


In [4]:
train_loader = DataLoader(train_dataset,batch_size=args.batch_size,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=args.batch_size,shuffle=False)

In [5]:
# Defining the autoencoder
hidden_dim = 128
latent_dim = 12
n_layers = 3 

autoencoder = AutoEncoder(n_dim,hidden_dim,latent_dim,n_layers).to(args.device)

In [6]:
# Defining the generator, discriminator and regressor

n_channel = args.n_channel
generator = LieGenerator(latent_dim,n_channel,args).to(args.device)

d_input_size = latent_dim*(input_timesteps + output_timesteps)
discriminator = LieDiscriminator(d_input_size).to(args.device)

regressor = PredModel(
    n_dim = latent_dim, 
    input_dim = input_timesteps, 
    hidden_dim = hidden_dim, 
    output_dim = output_timesteps
).to(args.device)

In [7]:
w_recon = 2
w_gan = 1
w_reg = 1e-2 
w_chreg = 0 
w_reg_z = 1e-3
w_reg_x = 1
log_interval = 1
save_interval = 100 

reg_type = args.reg_type

num_epochs = 500
batch_size = args.batch_size
lr_ae = 1e-3 
lr_g = 1e-3
lr_d = 1e-3 
lr_reg = 1e-3

device = get_device()
save_dir = 'LaLiGAN/vanilla/two_body'

In [8]:
flatten_input = True 

stats = train_lassi(
   autoencoder, discriminator, generator, train_loader, test_loader,
    num_epochs, lr_ae, lr_d, lr_g, w_recon, w_gan, reg_type, w_reg, w_chreg,
    regressor, lr_reg, w_reg_z, w_reg_x,
    device, save_interval, save_dir
)

Starting training: 500 epochs


  0%|          | 1/500 [00:01<08:29,  1.02s/it]

Epoch 1/500, Train Loss: 0.156088, Test Loss: 0.025564
loss_ae: 0.2546, loss_g: 0.3113, loss_g_reg: 0.9190, loss_g_chreg: 0.0000, loss_d_real: 0.3084, loss_d_fake: 1.3761, loss_ae_rel: 0.4184
Parameter containing:
tensor([[[-0.0165,  0.0462,  0.3002, -0.0938, -0.2094, -0.2357, -0.1411,
          -0.1387, -0.1058,  0.1791, -0.1322,  0.0379],
         [ 0.1817,  0.1127, -0.0641,  0.1484, -0.0137,  0.0102,  0.1365,
           0.1375,  0.1003,  0.0029, -0.0727,  0.0623],
         [ 0.0274, -0.1026,  0.1089, -0.0350,  0.1145,  0.0360,  0.1309,
           0.0781, -0.0077,  0.1317, -0.0958, -0.0090],
         [ 0.0851, -0.0102, -0.3057, -0.0199, -0.0019,  0.0330,  0.1611,
           0.2132, -0.1382,  0.1605,  0.1895,  0.0290],
         [ 0.1638,  0.0568, -0.0107,  0.1192,  0.0622,  0.0945, -0.0163,
           0.1095,  0.0537,  0.1095, -0.1306, -0.0975],
         [ 0.0728, -0.0467,  0.1052,  0.0803, -0.3388,  0.0077, -0.1102,
          -0.0524, -0.0717, -0.0953, -0.0081,  0.1047],
         [ 0

  2%|▏         | 11/500 [00:06<04:33,  1.79it/s]

Epoch 11/500, Train Loss: 0.018986, Test Loss: 0.005654
loss_ae: 0.0237, loss_g: 0.2972, loss_g_reg: 0.9901, loss_g_chreg: 0.0000, loss_d_real: 0.2969, loss_d_fake: 1.3744, loss_ae_rel: 0.0387
Parameter containing:
tensor([[[ 0.0425,  0.1805,  0.2027, -0.0244, -0.2217, -0.1202, -0.0898,
          -0.1865, -0.2210,  0.1237, -0.1107,  0.0863],
         [ 0.1001,  0.1104, -0.0533,  0.1147, -0.0587,  0.0615,  0.1473,
           0.0176,  0.1497,  0.0520, -0.0165,  0.0549],
         [ 0.0861,  0.0097,  0.1634, -0.0331,  0.0713,  0.1688,  0.0961,
           0.0278, -0.0527,  0.0357, -0.0003,  0.0012],
         [ 0.0966,  0.0379, -0.2027,  0.0558, -0.1336,  0.1377,  0.0106,
           0.2576, -0.0433,  0.0618,  0.1885, -0.0041],
         [ 0.0933, -0.0469, -0.0190,  0.1421,  0.1105,  0.0220,  0.0381,
          -0.0152,  0.0567,  0.1512, -0.0389, -0.0531],
         [-0.0355, -0.0904,  0.0603,  0.0810, -0.2380,  0.0702, -0.0484,
          -0.1270, -0.1092, -0.0150,  0.0587,  0.0518],
         [-

...

Epoch 481/500, Train Loss: 0.004663, Test Loss: 0.002142
loss_ae: 0.0046, loss_g: 0.2884, loss_g_reg: 1.0000, loss_g_chreg: 0.0000, loss_d_real: 0.2883, loss_d_fake: 1.3863, loss_ae_rel: 0.0075
Parameter containing:
tensor([[[-0.0586, -0.0079,  0.1002, -0.0532, -0.0685, -0.0865,  0.0369,
          -0.1334, -0.1788,  0.0579, -0.0591,  0.1881],
         [ 0.0386, -0.0137, -0.0101, -0.0763,  0.0093,  0.1851, -0.0368,
          -0.0364, -0.0867,  0.0029,  0.0765,  0.2719],
         [ 0.1447,  0.1799, -0.1739,  0.2605, -0.1692,  0.0660, -0.1175,
           0.2245,  0.1163,  0.0933,  0.0795, -0.2118],
         [-0.0485, -0.1049,  0.0913,  0.2701,  0.1442, -0.0013,  0.0350,
           0.2192,  0.1281,  0.1167,  0.0825,  0.0655],
         [-0.0470,  0.0839, -0.0350, -0.2386,  0.0540,  0.0654, -0.0395,
          -0.1382,  0.0650, -0.0294,  0.2541, -0.1977],
         [-0.1255,  0.0655, -0.1628,  0.0237, -0.1346, -0.0401,  0.1245,
           0.1052,  0.0040,  0.0753, -0.2238,  0.1148],
         [

 98%|█████████▊| 491/500 [04:08<00:04,  2.18it/s]

Epoch 491/500, Train Loss: 0.003941, Test Loss: 0.001347
loss_ae: 0.0033, loss_g: 0.2885, loss_g_reg: 0.9999, loss_g_chreg: 0.0000, loss_d_real: 0.2894, loss_d_fake: 1.3857, loss_ae_rel: 0.0054
Parameter containing:
tensor([[[-0.0129, -0.0412,  0.1061, -0.0378, -0.0464, -0.1174,  0.0093,
          -0.0811, -0.1702,  0.0422, -0.0873,  0.1866],
         [ 0.0519, -0.0034, -0.0188, -0.0936,  0.0143,  0.1932, -0.0444,
          -0.0260, -0.0890, -0.0227,  0.0537,  0.2591],
         [ 0.1313,  0.1656, -0.1555,  0.2976, -0.1765,  0.0540, -0.1076,
           0.2455,  0.1234,  0.0988,  0.0742, -0.2235],
         [-0.0497, -0.0631,  0.1086,  0.2392,  0.0665,  0.0621,  0.0052,
           0.1320,  0.1179,  0.0795,  0.1818,  0.1283],
         [-0.0648,  0.0818, -0.0072, -0.2384,  0.0262,  0.0835, -0.0498,
          -0.1627,  0.0793, -0.0304,  0.2830, -0.1929],
         [-0.1583,  0.0492, -0.1516,  0.0520, -0.1395, -0.0516,  0.1667,
           0.1060, -0.0098,  0.1196, -0.2634,  0.0810],
         [

100%|██████████| 500/500 [04:13<00:00,  1.98it/s]

Epoch 500/500, Train Loss: 0.004030, Test Loss: 0.001919
loss_ae: 0.0034, loss_g: 0.2886, loss_g_reg: 0.9999, loss_g_chreg: 0.0000, loss_d_real: 0.2881, loss_d_fake: 1.3912, loss_ae_rel: 0.0055
Parameter containing:
tensor([[[-0.0057, -0.0427,  0.0488, -0.0419,  0.0342, -0.1421,  0.0513,
          -0.0519, -0.1641,  0.0529, -0.0357,  0.1467],
         [ 0.0932, -0.0447,  0.0025, -0.0901,  0.0234,  0.1471, -0.0881,
           0.0491, -0.0329, -0.0361,  0.0295,  0.2377],
         [ 0.1440,  0.1245, -0.0934,  0.3307, -0.2303,  0.0423, -0.1238,
           0.2654,  0.1331,  0.0910,  0.0034, -0.2420],
         [-0.0422, -0.0668,  0.1004,  0.2060,  0.0547,  0.0770,  0.0240,
           0.1246,  0.1179,  0.0782,  0.1888,  0.1342],
         [-0.0418,  0.1037, -0.0129, -0.2453,  0.0262,  0.0963, -0.0634,
          -0.1692,  0.0841, -0.0650,  0.2894, -0.1827],
         [-0.1344,  0.0651, -0.1033,  0.0297, -0.1858, -0.0290,  0.1194,
           0.0901,  0.0263,  0.0782, -0.3057,  0.1220],
         [

In [9]:
## Training LaLiGAN without regressor

autoencoder_ = AutoEncoder(n_dim,hidden_dim,latent_dim,n_layers).to(args.device)

n_channel = args.n_channel
generator = LieGenerator(latent_dim,n_channel,args).to(args.device)

d_input_size = latent_dim*(input_timesteps + output_timesteps)
discriminator = LieDiscriminator(d_input_size).to(args.device)

In [10]:
save_dir = 'LaLiGAN/EMLP/two_body'

w_recon = 1
w_gan = 2
w_reg = 1e-2 
w_chreg = 0 
log_interval = 1
save_interval = 100 

reg_type = args.reg_type

num_epochs = 500
batch_size = args.batch_size
lr_ae = 1e-3 
lr_g = 1e-3
lr_d = 1e-3 
lr_reg = 1e-3

train_laliegan(
  autoencoder_, discriminator, generator, train_loader,
    num_epochs, lr_ae, lr_d, lr_g, w_recon, w_gan, reg_type, w_reg, w_chreg,
    regressor, device, save_interval, save_dir
)

loss_ae: 0.1283, loss_g: 0.2055, loss_g_reg: 0.9171, loss_g_chreg: 0.0000, loss_d_real: 0.2154, loss_d_fake: 1.8370, loss_ae_rel: 0.4168
loss_ae: 0.0109, loss_g: 0.1864, loss_g_reg: 0.9760, loss_g_chreg: 0.0000, loss_d_real: 0.1868, loss_d_fake: 1.7826, loss_ae_rel: 0.0355
loss_ae: 0.0119, loss_g: 0.1895, loss_g_reg: 0.9855, loss_g_chreg: 0.0000, loss_d_real: 0.1896, loss_d_fake: 1.7784, loss_ae_rel: 0.0386
loss_ae: 0.0117, loss_g: 0.1901, loss_g_reg: 0.9891, loss_g_chreg: 0.0000, loss_d_real: 0.1834, loss_d_fake: 1.7865, loss_ae_rel: 0.0384
loss_ae: 0.0130, loss_g: 0.1864, loss_g_reg: 0.9888, loss_g_chreg: 0.0000, loss_d_real: 0.1859, loss_d_fake: 1.7823, loss_ae_rel: 0.0424
loss_ae: 0.0119, loss_g: 0.1848, loss_g_reg: 0.9892, loss_g_chreg: 0.0000, loss_d_real: 0.1832, loss_d_fake: 1.7915, loss_ae_rel: 0.0388
loss_ae: 0.0086, loss_g: 0.1846, loss_g_reg: 0.9946, loss_g_chreg: 0.0000, loss_d_real: 0.1843, loss_d_fake: 1.7905, loss_ae_rel: 0.0280


loss_ae: 0.0110, loss_g: 0.1836, loss_g_reg: 0.9954, loss_g_chreg: 0.0000, loss_d_real: 0.1843, loss_d_fake: 1.7957, loss_ae_rel: 0.0357
loss_ae: 0.0091, loss_g: 0.1843, loss_g_reg: 0.9968, loss_g_chreg: 0.0000, loss_d_real: 0.1855, loss_d_fake: 1.7931, loss_ae_rel: 0.0294
loss_ae: 0.0113, loss_g: 0.1851, loss_g_reg: 0.9993, loss_g_chreg: 0.0000, loss_d_real: 0.1855, loss_d_fake: 1.7923, loss_ae_rel: 0.0368
loss_ae: 0.0145, loss_g: 0.1848, loss_g_reg: 0.9976, loss_g_chreg: 0.0000, loss_d_real: 0.1843, loss_d_fake: 1.7939, loss_ae_rel: 0.0471
loss_ae: 0.0094, loss_g: 0.1847, loss_g_reg: 0.9995, loss_g_chreg: 0.0000, loss_d_real: 0.1843, loss_d_fake: 1.7940, loss_ae_rel: 0.0305
loss_ae: 0.0083, loss_g: 0.1841, loss_g_reg: 0.9990, loss_g_chreg: 0.0000, loss_d_real: 0.1847, loss_d_fake: 1.7909, loss_ae_rel: 0.0271
loss_ae: 0.0075, loss_g: 0.1836, loss_g_reg: 0.9993, loss_g_chreg: 0.0000, loss_d_real: 0.1836, loss_d_fake: 1.7921, loss_ae_rel: 0.0245
loss_ae: 0.0084, loss_g: 0.1829, loss_g_r

In [11]:
n_channel = 1

lie_algebra = get_generators(latent_dim,n_channel,f'saved_model/LaLiGAN/EMLP/two_body/laligan_generator_{num_epochs-1}.pt','pendulum')
G = CustomGroup(latent_dim,lie_algebra) 

In [ ]:
# 1. Extract weights from PyTorch model
torch_weights = extract_weights_from_torch(autoencoder_)

# 2. Create new Objax model
jax_autoencoder = JAXAutoEncoder(input_dim=n_dim, hidden_dim=hidden_dim, latent_dim=latent_dim, n_layers=n_layers, batch_norm=True)

# 3. Load weights
load_weights_into_jax(jax_autoencoder, torch_weights)

✅ All weights transferred successfully.


In [13]:
lr = 1e-3

model_emlp, stats_emlp = train_predModel_laligan(
    G, jax_autoencoder, train_dataset, test_dataset,
    batch_size, num_epochs, lr, latent_dim, input_timesteps, output_timesteps
)

E0607 21:15:36.002985 2425617 hlo_lexer.cc:443] Failed to parse int literal: 986347166337587168698000264
E0607 21:15:36.003027 2425617 hlo_lexer.cc:443] Failed to parse int literal: 986347166337587168698000264


Starting training: 500 epochs


  0%|          | 1/500 [00:19<2:40:43, 19.33s/it]

Epoch 1/500, Train Loss: 0.421749, Test Loss: 0.427011


  2%|▏         | 11/500 [00:21<03:26,  2.37it/s] 

...

Epoch 481/500, Train Loss: 0.002375, Test Loss: 0.002240


 98%|█████████▊| 491/500 [02:24<00:02,  4.07it/s]

Epoch 491/500, Train Loss: 0.002368, Test Loss: 0.002228


100%|██████████| 500/500 [02:26<00:00,  3.41it/s]

Epoch 500/500, Train Loss: 0.002368, Test Loss: 0.002201


In [ ]:
# Plot training statistics
plt.figure(figsize=(15, 7))

plt.subplot(2, 1, 1)
plt.plot(stats['train_loss'],label='LaLiGAN')
plt.plot(stats_emlp['train_loss'],label='EMLP LaLiGAN')
plt.yscale('log')
plt.title('Train Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(stats['test_loss'],label='LaLiGAN')
plt.plot(stats_emlp['test_loss'],label='EMLP LaLiGAN')
plt.yscale('log')
plt.title('Test Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.grid(True)

plt.suptitle('LaLiGAN MultiStep Prediction Training Statistics')
plt.tight_layout()
plt.show()



In [15]:
print('Loading Two Body Dynamics dataset...') 
with open('data/two_body_dataset.pkl','rb') as file:
    data_dict = pickle.load(file)

test_coords = data_dict['test_coords']

Loading Two Body Dynamics dataset...


In [16]:
def forecast_emlp_laligan(model, autoencoder, last_sequence, n_steps, feature_dim, latent_dim):
    devices = jax.devices("gpu")
    if devices:
        device = devices[0]
        sequence = jax.device_put(jnp.array(last_sequence)[None, ...], device)
    else:
        sequence = jnp.array(last_sequence)[None, ...]

    # Encode the input sequence to latent space
    current_sequence_enc = autoencoder.encode(sequence, training=False)

    # JIT compile the model
    model_jit = jax.jit(model)

    # Run a forward pass to get forecast horizon
    sample_pred = model_jit(current_sequence_enc).reshape(1,-1,latent_dim)
    forecast_horizon = sample_pred.shape[1]
    # Prepare forecast array in observation space

    forecast = np.zeros((n_steps, feature_dim))



    for i in range(0, n_steps, forecast_horizon):
        next_steps_enc = model_jit(current_sequence_enc).reshape(-1,forecast_horizon,latent_dim)
        if next_steps_enc.ndim == 2:
            next_steps_enc = next_steps_enc[:, None, :]

        # Decode to observation space
        next_steps = autoencoder.decode(next_steps_enc, training=False)
        next_steps_np = np.array(next_steps[0])  # Remove batch dimension

        # Store into forecast
        steps_to_add = min(forecast_horizon, n_steps - i)
        forecast[i:i + steps_to_add] = next_steps_np[:steps_to_add]

        if i + forecast_horizon < n_steps:
            # Re-encode next steps for next iteration
            next_steps_jax = jnp.array(next_steps_np[:steps_to_add])
            if devices:
                next_steps_jax = jax.device_put(next_steps_jax, device)

            next_steps_enc = autoencoder.encode(next_steps_jax[None, ...], training=False)

            # Shift and append to form new input sequence
            current_sequence_enc = jnp.concatenate([
                current_sequence_enc[:, forecast_horizon:, :],
                next_steps_enc
            ], axis=1)

    return forecast

In [17]:
idx = np.random.choice(range(0,len(test_coords)))
last_sequence = test_coords[idx,:input_timesteps,:]
n_steps = len(test_coords[0,:,:])-len(last_sequence)

forecast = forecast_nn_laligan(regressor,autoencoder,last_sequence,n_steps,n_dim,latent_dim)
forecast_e = forecast_emlp_laligan(model_emlp,jax_autoencoder,last_sequence,n_steps,n_dim,latent_dim)

In [18]:
def plot_forecast_comparison(forecast, forecast_e, test_coords, input_timesteps):
    """
    Plot all 8 variables from four different forecast models for comparison.
    
    Parameters:
    -----------
    forecast : array - Baseline model predictions
    forecast_e : array - EMLP model predictions
    test_coords : array - Ground truth coordinates
    input_timesteps : int - Number of initial timesteps used for prediction
    """
    # Extract ground truth data for comparison
    n_steps = len(forecast)
    ground_truth = test_coords[0, input_timesteps:input_timesteps+n_steps]
    
    # Create time steps for x-axis
    time_steps = np.arange(n_steps)
    
    # Variable names for labels (assuming a two-body system like in the example)
    var_names = ['x₁', 'y₁', 'x₂', 'y₂', 'p_x₁', 'p_y₁', 'p_x₂', 'p_y₂']
    
    # Define a consistent color scheme
    colors = {
        'ground_truth': '#d62728',  # Red
        'baseline': '#1f77b4',      # Blue
        'aug_vanilla': '#9467bd',   # Purple
        'aug_improved': '#2ca02c',  # Green
        'emlp': '#ff7f0e'           # Orange
    }
    
    # Create a 4x2 grid of subplots
    fig, axes = plt.subplots(4, 2, figsize=(15, 16))
    fig.suptitle('Comparison of Two-Body System State Variables Forecasts', fontsize=16)
    
    # Plot all 8 variables
    for i, ax in enumerate(axes.flat):
        # Plot predictions from all models
        ax.plot(time_steps, ground_truth[:, i], '--', color=colors['ground_truth'], 
                label='Ground Truth', linewidth=2, alpha=0.8)
        ax.plot(time_steps, forecast[:, i], '-', color=colors['baseline'], 
                label='LaLiGAN', linewidth=1.5)
        ax.plot(time_steps, forecast_e[:, i], '-', color=colors['emlp'], 
                label='LaLiGAN EMLP', linewidth=1.5)
        
        ax.set_title(f'{var_names[i]}', fontsize=12)
        ax.set_xlabel('Time Steps', fontsize=10)
        ax.set_ylabel('Value', fontsize=10)
        ax.grid(True, alpha=0.3)
        
        # Only add legend to the first plot to save space
        if i == 0:
            ax.legend(fontsize=10)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust for suptitle
    
    # Plot orbital trajectories of both bodies
    plt.figure(figsize=(12, 10))
    plt.title('Orbital Trajectories Comparison', fontsize=16)
    
    # Body 1 trajectories (positions x₁, y₁)
    plt.plot(ground_truth[:, 0], ground_truth[:, 1], '--', color=colors['ground_truth'], 
             label='Body 1 (Ground Truth)', linewidth=2.5, alpha=0.8)
    plt.plot(forecast[:, 0], forecast[:, 1], '-', color=colors['baseline'], 
             label='Body 1 (Baseline)', linewidth=1.5)
    plt.plot(forecast_e[:, 0], forecast_e[:, 1], '-', color=colors['emlp'], 
             label='Body 1 (EMLP)', linewidth=1.5)
    
    # Body 2 trajectories (positions x₂, y₂)
    plt.plot(ground_truth[:, 2], ground_truth[:, 3], ':', color=colors['ground_truth'], 
             label='Body 2 (Ground Truth)', linewidth=2.5, alpha=0.8)
    plt.plot(forecast[:, 2], forecast[:, 3], ':', color=colors['baseline'], 
             label='Body 2 (Baseline)', linewidth=1.5)
    plt.plot(forecast_e[:, 2], forecast_e[:, 3], ':', color=colors['emlp'], 
             label='Body 2 (EMLP)', linewidth=1.5)
    
    plt.xlabel('x position', fontsize=12)
    plt.ylabel('y position', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.axis('equal')
    plt.legend(fontsize=10, loc='best')
    
    # Plot energy conservation (assuming Hamiltonian system)
    plt.figure(figsize=(12, 6))
    plt.title('Total Energy Over Time', fontsize=16)
    
    # Calculate approximate energy (kinetic + potential) if this is a Hamiltonian system
    # For simplicity, assuming masses are 1 and only considering kinetic energy from momenta
    # E ≈ (p_x₁² + p_y₁² + p_x₂² + p_y₂²)/2
    def approx_energy(data):
        return 0.5 * (data[:, 4]**2 + data[:, 5]**2 + data[:, 6]**2 + data[:, 7]**2)
    
    plt.plot(time_steps, approx_energy(ground_truth), '--', color=colors['ground_truth'], 
             label='Ground Truth', linewidth=2.5, alpha=0.8)
    plt.plot(time_steps, approx_energy(forecast), '-', color=colors['baseline'], 
             label='Baseline MLP', linewidth=1.5)
    plt.plot(time_steps, approx_energy(forecast_e), '-', color=colors['emlp'], 
             label='EMLP', linewidth=1.5)
    
    plt.xlabel('Time Steps', fontsize=12)
    plt.ylabel('Approximate Energy', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=10, loc='best')
    
    plt.tight_layout()
    plt.show()

# Call the function with your data
# plot_forecast_comparison(
#     forecast[:50,:], 
#     forecast_aug_v[:50,:], 
#     forecast_aug_i[:50,:], 
#     forecast_e[:50,:], 
#     test_coords[:50,:], 
#     input_timesteps, 
# )

plot_forecast_comparison(
    forecast, 
    forecast_e, 
    test_coords, 
    input_timesteps, 
)

In [ ]:
## Computing Error metrics 

n = 10  # Number of samples
indices = np.random.choice(test_coords.shape[0], min(n,test_coords.shape[0]), replace=False)

def mse_comp(arr1,arr2):
    assert arr1.shape == arr2.shape 

    squared_diff = (arr1-arr2)**2 
    return np.mean(squared_diff)

mse = {
    'LaLiGAN':[],
    'EMLP LaLiGAN':[],
}

for i,idx in enumerate(indices):
    print(f'Trial {i+1}')
    sim = test_coords[idx,input_timesteps:,:]

    last_sequence = test_coords[idx,:input_timesteps,:]
    n_steps = len(test_coords[idx,:,:])-len(last_sequence)

    nn_forecast = forecast_nn_laligan(regressor,autoencoder,last_sequence,n_steps,n_dim,latent_dim)
    emlp_forecast = forecast_emlp_laligan(model_emlp,jax_autoencoder,last_sequence,n_steps,n_dim,latent_dim)


    mse['LaLiGAN'].append(mse_comp(sim,nn_forecast))
    mse['EMLP LaLiGAN'].append(mse_comp(sim,emlp_forecast))




Trial 1
Trial 2
Trial 3
Trial 4
Trial 5
Trial 6
Trial 7
Trial 8
Trial 9
Trial 10


In [20]:
for k in mse.keys():
    print(f'{k} : {np.mean(mse[k])}')

LaLiGAN : 0.3150063468476983
EMLP LaLiGAN : 0.5377974898237593
